In [ ]:
import json
import csv
import os
from datetime import datetime

# The directory containing your folders with JSON files
root_directory = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\track_information'

# The path for the output CSV file
csv_file_path = 'tracks_data.csv'

# Current date for 'Date Added To DB' and 'Date Last Modified' columns in the YYYY-MM-DD format
current_date = datetime.now().strftime('%Y-%m-%d')

# Template for CSV rows with the updated header titles
csv_columns = [
    'id', 'song_spotify_id', 'song_title', 'song_duration', 'song_album_type', 
    'song_album_id', 'song_explicit', 'song_popularity', 'song_preview_url', 
    'song_track_features_added', 'song_acousticness', 'song_danceability', 'song_energy', 
    'song_instrumentalness', 'song_liveness', 'song_loudness', 'song_speechiness', 
    'song_tempo', 'song_valence', 'song_key', 'song_time_signature', 'song_date_added_to_db', 
    'song_date_last_modified'
]

# Initialize an empty list to store rows for the CSV
csv_rows = []

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for track in data.get('tracks', []):
            # Skip processing if the track is None
            if track is None:
                continue

            row = {
                'id': -1,  # This will be updated later with the actual ID
                'song_spotify_id': track['id'],
                'song_title': track['name'],
                'song_duration': track['duration_ms'],
                'song_album_type': track['album']['album_type'].upper(),
                'song_album_id': track['album']['id'],
                'song_explicit': track['explicit'],
                'song_popularity': track['popularity'],
                'song_preview_url': track.get('preview_url', ''),
                'song_track_features_added': False,
                'song_acousticness': -1,
                'song_danceability': -1,
                'song_energy': -1,
                'song_instrumentalness': -1,
                'song_liveness': -1,
                'song_loudness': -1,
                'song_speechiness': -1,
                'song_tempo': -1,
                'song_valence': -1,
                'song_key': -1,
                'song_time_signature': -1,
                'song_date_added_to_db': current_date,
                'song_date_last_modified': current_date
            }
            csv_rows.append(row)

# Iterate over each subfolder and JSON file in the directory
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        if filename.endswith('.json'):
            print(f"Processing {filename}...")
            process_json_file(os.path.join(subdir, filename))

# Write the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns, delimiter=';')
    writer.writeheader()
    
    # Update each row with its ID before writing
    for i, row in enumerate(csv_rows, start=1):
        row['id'] = i
        writer.writerow(row)

print(f"CSV file has been successfully created at {csv_file_path} with {len(csv_rows)} tracks.")


In [ ]:
import json
import csv
import os
from datetime import datetime

# Define the root directory containing your JSON files
root_directory = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\ALBUMS'

# Define the output CSV file path
csv_file_path = 'album_data.csv'

# Current date for 'Date Added To DB' and 'Date Last Modified' columns
current_date = datetime.now().strftime('%Y-%m-%d')

# CSV column headers
csv_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art', 'album_release_date',
    'release_date_precision', 'album_popularity', 'album_type', 'spotify_album_upc',
    'spotify_album_ean', 'spotify_album_isrc', 'date_added_to_db', 'date_last_modified',
    'musicbrainz_metadata_added', 'musicbrainz_id'
]

# Initialize a list to hold album data
albums_data = []

# Function to process each JSON file
def process_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        for album in data.get('albums', []):
            # Extract the required information, with checks for nullable fields
            album_data = {
                'id': -1,  # Placeholder, will be updated later
                'album_spotify_id': album['id'],
                'album_name': album['name'],
                'album_cover_art': album['images'][0]['url'] if album.get('images') else '',
                'album_release_date': album['release_date'],
                'release_date_precision': album['release_date_precision'],
                'album_popularity': album['popularity'],
                'album_type': album['album_type'],
                'spotify_album_upc': album['external_ids'].get('upc', '') if album.get('external_ids') else '',
                'spotify_album_ean': album['external_ids'].get('ean', '') if album.get('external_ids') else '',
                'spotify_album_isrc': album['external_ids'].get('isrc', '') if album.get('external_ids') else '',
                'date_added_to_db': current_date,
                'date_last_modified': current_date,
                'musicbrainz_metadata_added': False,  # Placeholder
                'musicbrainz_id': ''  # Placeholder
            }
            albums_data.append(album_data)

# Process each JSON file in the directory and subdirectories
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        if filename.endswith('.json'):
            process_json_file(os.path.join(subdir, filename))

# Write the data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns, delimiter=';')
    writer.writeheader()
    
    # Update each row with its actual ID before writing
    for i, album_data in enumerate(albums_data, start=1):
        album_data['id'] = i
        writer.writerow(album_data)

print(f"CSV file has been successfully created at {csv_file_path} with {len(albums_data)} albums.")


In [ ]:
import pandas as pd

# Specify the dtypes for the IDs to be strings when reading the CSVs
dtype_dict = {'id': str, 'song_album_id': str, 'album_spotify_id': str}
tracks_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\tracks_data.csv', delimiter=';', dtype=dtype_dict)
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\album_data.csv', delimiter=';', dtype=dtype_dict)

# Create a dictionary mapping from album_spotify_id to id from albums_df
# Ensure the 'id' column in albums_df is converted to integer if it's not NaN
album_id_map = albums_df.dropna(subset=['id']).set_index('album_spotify_id')['id'].astype(int).to_dict()

# Map the song_album_id in tracks_df using the album_id_map to get the album id
tracks_df['album_id'] = tracks_df['song_album_id'].map(album_id_map)

# Convert the new album_id column to integers, NaNs will be converted to a float with a .0
tracks_df['album_id'] = tracks_df['album_id'].fillna(-1).astype(int)

# Replace -1 back to NaN if you want to keep NaN values
tracks_df['album_id'].replace(-1, pd.NA, inplace=True)

# Save the updated tracks DataFrame to a new CSV file
tracks_df.to_csv('path_to_updated_tracks.csv', index=False, sep=';')


In [ ]:



import pandas as pd
import os
import json

# Read the albums and artists CSVs into DataFrames
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Dictionary to map Spotify album ID to CSV album ID
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()

# Dictionary to map Spotify artist ID to CSV artist ID
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

# Initialize a list to hold the artist-album mappings
artist_album_mapping = []

# Assuming 'path_to_json_folder' is the folder containing all the JSON subfolders
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if album_id:
                        for artist in album['artists']:
                            artist_id = artist_id_map.get(artist['id'])
                            if artist_id:
                                artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Create a DataFrame from the artist-album mappings
artist_album_df = pd.DataFrame(artist_album_mapping)

# Remove duplicates if there are any
artist_album_df = artist_album_df.drop_duplicates()

# Save the DataFrame to CSV
artist_album_df.to_csv('artist_album_mappings_new.csv', index=False, sep=';')


In [ ]:
import pandas as pd
import os
import json

# Read the albums and artists CSVs into DataFrames
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\album_data.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Dictionary to map Spotify album ID to CSV album ID
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()

# Dictionary to map Spotify artist ID to CSV artist ID
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

# Initialize a list to hold the artist-album mappings
artist_album_mapping = []

# List to hold artist Spotify IDs where CSV artist ID was not found
missing_artist_ids = []

# Assuming 'path_to_json_folder' is the folder containing all the JSON subfolders
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if album_id:
                        for artist in album['artists']:
                            artist_spotify_id = artist['id']
                            artist_id = artist_id_map.get(artist_spotify_id)
                            if artist_id:
                                artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})
                            else:
                                missing_artist_ids.append(artist_spotify_id)

# Print the list of artist Spotify IDs where the CSV artist ID was not found
print("List of artist Spotify IDs where the CSV artist ID was not found:")
for artist_id in missing_artist_ids:
    print(artist_id)

In [ ]:
import pandas as pd

# Convert the list to a DataFrame
missing_artist_ids_df = pd.DataFrame(missing_artist_ids, columns=['missing_artist_spotify_id'])

# Define the file path where you want to save the CSV
file_path = 'missing_artist_ids.csv'  # You can specify your own path

# Save the DataFrame to a CSV file
missing_artist_ids_df.to_csv(file_path, index=False)

print(f'The missing artist IDs have been saved to {file_path}')

In [ ]:
import pandas as pd

# Replace 'input_file.csv' with the path to your input CSV file
input_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\musicBrainzAllContributors.csv'
# Replace 'output_file.csv' with the path where you want to save the output CSV file
output_file_path = 'contributor_data_prelink.csv'

# Step 1: Read the input CSV
input_df = pd.read_csv(input_file_path, delimiter=',')

# Step 2: Create the new DataFrame
output_df = pd.DataFrame({
    'ID': range(1, len(input_df) + 1),
    'NAME': input_df['artist_credit_name'],
    'ROLE': input_df['role'],
    'INSTRUMENT': input_df['instrument'],
    'MUSICBRAINZ_ID': input_df['artist_mbid'],
    'MAINARTIST': input_df['artist_credit_name'],
    'SONGTITLE': input_df['recording_name']
})

# Step 3: Write the resulting DataFrame to a new CSV file
output_df.to_csv(output_file_path, sep=';', index=False)

print(f"Output CSV saved to {output_file_path}")


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Load the CSV files
musicbrainz_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\PythonNotebooksForPrepopulation\\contributor_data_prelink.csv', sep=';')  # The file generated from the previous step
spotify_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\tracks_data.csv', sep=';')  # The Spotify songs CSV

# Create a list of combined song title and artist names from Spotify for matching
spotify_combined_list = spotify_df.apply(lambda x: f"{x['song_title']} {x['song_album_id']}", axis=1).tolist()

def find_best_match(mb_row, spotify_combined_list):
    mb_combined = f"{mb_row['SONGTITLE']} {mb_row['MAINARTIST']}"
    
    # Using extractOne to find the best match from the list
    best_match_info = process.extractOne(mb_combined, spotify_combined_list)
    
    # If there is a match found, extract it
    if best_match_info:
        best_match_text, best_score = best_match_info
        # Find the index of the match in Spotify list to retrieve the full row from spotify_df
        match_index = spotify_combined_list.index(best_match_text)
        best_match_row = spotify_df.iloc[match_index]
        return best_match_row, best_score
    return None, 0

# Prepare the output DataFrame
matched_df = pd.DataFrame(columns=['CONTRIBUTOR_ID', 'SONG_TABLE_ID', 'Spotify_Song_Title', 'Spotify_Artist', 'MusicBrainz_Song_Title', 'MusicBrainz_Artist'])

# Iterate over MusicBrainz entries to find matches
for index, mb_row in musicbrainz_df.iterrows():
    best_match_row, score = find_best_match(mb_row, spotify_combined_list)
    if best_match_row is not None and score > 80:  # Adjust the threshold as needed
        matched_df = matched_df.append({
            'CONTRIBUTOR_ID': mb_row['ID'],
            'SONG_TABLE_ID': best_match_row['id'],
            'Spotify_Song_Title': best_match_row['song_title'],
            'Spotify_Artist': best_match_row['song_album_id'],  # Note: Adjust if there's a more direct artist name column
            'MusicBrainz_Song_Title': mb_row['SONGTITLE'],
            'MusicBrainz_Artist': mb_row['MAINARTIST']
        }, ignore_index=True)

# Output the matched DataFrame to CSV
matched_df.to_csv('linked_songs.csv', sep=';', index=False)

print("Linked songs CSV generated.")

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\contributors_non-normalized.csv')

# Select only the 'artist_credit_name' and 'artist_credit_id' columns
df_selected = df[['artist_credit_name', 'artist_credit_id']]

# Drop duplicates based on 'artist_credit_id' to ensure each ID is unique
df_unique = df_selected.drop_duplicates(subset=['artist_credit_id'])

# Save the result to a new CSV file
df_unique.to_csv('unique_artist_credits.csv', index=False)

print('Unique artist credits CSV file has been saved as unique_artist_credits.csv.')


In [ ]:
import pandas as pd

def parse_manual(array_str):
    # Manually parse the string to extract elements between curly braces
    # Remove leading and trailing braces and split by comma
    items = array_str.strip('{}').split(',')
    # Strip quotes and extra spaces from each item
    items = [item.strip('"').strip() for item in items]
    return items

# Load the CSV file
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\contributors_non-normalized.csv')

# Prepare a list to collect all artist name-MBID pairs
artist_pairs = []
i=0

# Iterate over the DataFrame rows
for _, row in df.iterrows():
    # Parse the 'artist_mbids' and 'individual_artist_names' fields manually
    artist_mbids = parse_manual(row['artist_mbids'])
    individual_artist_names = parse_manual(row['individual_artist_names'])
    
    # Ensure we have equal lengths of MBIDs and names before proceeding
    if len(artist_mbids) == len(individual_artist_names):
        # Pair each individual artist name with its corresponding MBID
        for artist_name, artist_mbid in zip(individual_artist_names, artist_mbids):
            artist_pairs.append((artist_name, artist_mbid))
    else:
        print("Warning: Mismatched MBIDs and artist names for a row, skipping.")
        print(f"Row index: {i}")
        i = i+1

# Convert the list of pairs into a DataFrame, ensuring uniqueness
df_pairs = pd.DataFrame(list(set(artist_pairs)), columns=['individual_artist_name', 'artist_mbid'])

# Save the DataFrame to a new CSV file
df_pairs.to_csv('artist_name_mbid_pairs.csv', index=False)

print('Artist name-MBID pairs CSV file has been saved as artist_name_mbid_pairs.csv.')



In [ ]:
import pandas as pd

# Load the MBID CSV
mbid_df = pd.read_csv('C:\\Users\\Music\\team_project\\team37\\prepopulationStuff\\PythonNotebooksForPrepopulation\\artist_name_mbid_pairs.csv')

# Load the Spotify artist CSV, remember to use the ';' delimiter
spotify_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data.csv', delimiter=';')

# Perform a direct merge based on artist names
# Note: This assumes 'artist_name' in spotify_df exactly matches 'individual_artist_name' in mbid_df
merged_df = pd.merge(spotify_df, mbid_df, how='left', left_on='artist_name', right_on='individual_artist_name')

# Drop the 'individual_artist_name' column as it's redundant after merge
merged_df.drop(columns=['individual_artist_name'], inplace=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('spotify_artist_with_mbid_direct_match.csv', index=False)

print('Spotify artist data with direct match MBIDs has been saved as spotify_artist_with_mbid_direct_match.csv.')


In [ ]:
import pandas as pd

# Adjust the path to your Spotify artists CSV file
spotify_csv_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\artists_data.csv'
spotify_df = pd.read_csv(spotify_csv_path, delimiter=';')

# Print column names to verify
print(spotify_df.columns)


In [ ]:
import pandas as pd
from datetime import datetime

# Load the CSV file
csv_file_path = 'C:\\Users\\Music\\team_project\\team37\\downloaded_files\\artists_data_full.csv'
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in columns that shouldn't have them
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to check data types
def check_data_types(df):
    errors = []
    for column, expected_type in expected_column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                errors.append(f"Column {column} is not of type {expected_type}")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                errors.append(f"Column {column} is not of type {expected_type}")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
            except ValueError:
                errors.append(f"Column {column} contains non-date values")
    
    if errors:
        for error in errors:
            print(error)
    else:
        print("All columns match their expected data types.")

# Columns expected not to have empty values based on your schema (all of them in this case)
mandatory_columns = [
    'id', 'artist_spotify_id', 'artist_name', 'artist_popularity',
    'artist_image_small', 'artist_image_medium', 'artist_image_large',
    'artist_followers', 'date_added_to_db', 'date_last_modified'
]

# Expected data types
expected_column_types = {
    'id': 'numeric',
    'artist_spotify_id': 'string',
    'artist_name': 'string',
    'artist_popularity': 'numeric',
    'artist_image_small': 'string',
    'artist_image_medium': 'string',
    'artist_image_large': 'string',
    'artist_followers': 'numeric',
    'date_added_to_db': 'date',
    'date_last_modified': 'date'
}

# Perform checks
check_empty_values(df, mandatory_columns)
check_data_types(df)


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', sep=';')  # Adjust the path and separator as needed

# Convert the dates in the last two columns
# Assuming the last two columns contain your dates
df.iloc[:, -4] = pd.to_datetime(df.iloc[:, -4], dayfirst=True).dt.strftime('%Y-%m-%d')
df.iloc[:, -3] = pd.to_datetime(df.iloc[:, -3], dayfirst=True).dt.strftime('%Y-%m-%d')

# Save the modified DataFrame back to a new CSV file
df.to_csv('path_to_modified_csv.csv', index=False, sep=';')  # Adjust the path and separator as needed


In [ ]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types
def check_data_types(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
                print(f"Column {column} is of correct date type.")
            except ValueError:
                print(f"Column {column} contains incorrect date format.")

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art',
    'album_release_date', 'release_date_precision', 'album_popularity',
    'album_type', 'date_added_to_db', 'date_last_modified', 'musicbrainz_metadata_added'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'album_spotify_id': 'string',
    'album_name': 'string',
    'album_cover_art': 'string',
    'album_release_date': 'date',
    'release_date_precision': 'string',
    'album_popularity': 'numeric',
    'album_type': 'string',
    'spotify_album_upc': 'string',  # Nullable
    'spotify_album_ean': 'string',  # Nullable
    'spotify_album_isrc': 'string',  # Nullable
    'date_added_to_db': 'date',
    'date_last_modified': 'date',
    'musicbrainz_metadata_added': 'boolean',  # Assuming true/false representation
    'musicbrainz_id': 'string'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct
check_data_types(df, column_types)
31045;59


In [ ]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv'

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types and print incorrect date formats
def check_data_types_and_dates(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            incorrect_format = df[column].apply(lambda x: check_date_format(x))
            if incorrect_format.any():
                print(f"Rows with incorrect date format in column '{column}':")
                print(df[incorrect_format])
            else:
                print(f"Column {column} is of correct date type.")

# Helper function to check date format
def check_date_format(date_string):
    try:
        pd.to_datetime(date_string, errors='raise')
        return False  # Date format is correct
    except ValueError:
        return True  # Date format is incorrect

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'album_spotify_id', 'album_name', 'album_cover_art',
    'album_release_date', 'release_date_precision', 'album_popularity',
    'album_type', 'date_added_to_db', 'date_last_modified', 'musicbrainz_metadata_added'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'album_spotify_id': 'string',
    'album_name': 'string',
    'album_cover_art': 'string',
    'album_release_date': 'date',
    'release_date_precision': 'string',
    'album_popularity': 'numeric',
    'album_type': 'string',
    'spotify_album_upc': 'string',  # Nullable
    'spotify_album_ean': 'string',  # Nullable
    'spotify_album_isrc': 'string',  # Nullable
    'date_added_to_db': 'date',
    'date_last_modified': 'date',
    'musicbrainz_metadata_added': 'boolean',  # Assuming true/false representation
    'musicbrainz_id': 'string'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct and for incorrect date formats
check_data_types_and_dates(df, column_types)


In [ ]:
import pandas as pd

# Load the CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv'  # Update this to the path of your CSV file

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')  # Ensure delimiter matches your CSV format

# Function to check for empty values in mandatory columns
def check_empty_values(df, columns):
    for column in columns:
        if df[column].isnull().any():
            print(f"Empty values found in column: {column}")
        else:
            print(f"No empty values in column: {column}")

# Function to verify data types
def check_data_types(df, column_types):
    for column, expected_type in column_types.items():
        if expected_type == 'numeric':
            if not pd.api.types.is_numeric_dtype(df[column]):
                print(f"Column {column} is not of numeric type.")
            else:
                print(f"Column {column} is of correct numeric type.")
        elif expected_type == 'string':
            if not pd.api.types.is_string_dtype(df[column]):
                print(f"Column {column} is not of string type.")
            else:
                print(f"Column {column} is of correct string type.")
        elif expected_type == 'date':
            try:
                pd.to_datetime(df[column])
                print(f"Column {column} is of correct date type.")
            except ValueError:
                print(f"Column {column} contains incorrect date format.")
        elif expected_type == 'boolean':
            # Checking if the column is boolean; Assuming boolean is represented as True/False or 1/0
            if not pd.api.types.is_bool_dtype(df[column]) and not all(df[column].dropna().isin([0, 1, 'True', 'False', True, False])):
                print(f"Column {column} is not of boolean type.")
            else:
                print(f"Column {column} is of correct boolean type.")

# Mandatory columns (not nullable)
mandatory_columns = [
    'id', 'song_spotify_id', 'song_title', 'song_duration', 
    'song_album_type', 'song_album_id', 'song_explicit', 
    'song_popularity', 'song_track_features_added', 'song_date_added_to_db', 
    'song_date_last_modified'
]

# Columns with specific data types
column_types = {
    'id': 'numeric',
    'song_spotify_id': 'string',
    'song_title': 'string',
    'song_duration': 'numeric',
    'song_album_type': 'string',
    'song_album_id': 'string',
    'song_explicit': 'boolean',
    'song_popularity': 'numeric',
    'song_preview_url': 'string',  # Nullable
    'song_track_features_added': 'boolean',
    # Assuming 'floatType' corresponds to 'numeric' in Python/Pandas
    'song_acousticness': 'numeric',  # Nullable
    'song_danceability': 'numeric',  # Nullable
    'song_energy': 'numeric',  # Nullable
    'song_instrumentalness': 'numeric',  # Nullable
    'song_liveness': 'numeric',  # Nullable
    'song_loudness': 'numeric',  # Nullable
    'song_speechiness': 'numeric',  # Nullable
    'song_tempo': 'numeric',  # Nullable
    'song_valence': 'numeric',  # Nullable
    'song_key': 'numeric',  # Nullable
    'song_time_signature': 'numeric',  # Nullable
    'song_date_added_to_db': 'date',
    'song_date_last_modified': 'date',
    'album_id': 'numeric'  # Nullable
}

# Check for empty values in mandatory columns
check_empty_values(df, mandatory_columns)

# Check if data types are correct
check_data_types(df, column_types)


In [ ]:
import pandas as pd

# Adjust these settings as needed
pd.set_option('display.max_rows', None)  # This will allow all rows to be displayed
pd.set_option('display.max_columns', None)  # This will allow all columns to be displayed
pd.set_option('display.width', 1000)  # Adjust the width to accommodate the number of columns
pd.set_option('display.max_colwidth', None)  # This ensures that the content of each column is fully displayed

# Define the path to your CSV file
csv_file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\tracks_data_full.csv'  # Update this to the path of your CSV file

# Load the CSV file
df = pd.read_csv(csv_file_path, delimiter=';')

# Function to print rows with empty values in a specific column
def print_rows_with_empty_values_in_column(df, column_name):
    empty_rows = df[df[column_name].isnull() | (df[column_name] == '')]
    if not empty_rows.empty:
        print(f"Rows with empty values in column '{column_name}':")
        # print the song_spotfiy_id for each row with empty values
        print(empty_rows['song_spotify_id'])
    else:
        print(f"No empty values found in column '{column_name}'.")

# Print all rows where the song_title is empty
print_rows_with_empty_values_in_column(df, 'song_title')



In [ ]:
import pandas as pd
import os
import json

# Load the CSV files
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_data_full.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Create dictionaries for ID mappings
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

artist_album_mapping = []

# Assuming the JSON structure is as shown in your example
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if not album_id:
                        print(f"Album ID {album['id']} not found in album CSV.")
                        continue
                    for artist in album['artists']:
                        artist_id = artist_id_map.get(artist['id'])
                        if not artist_id:
                            print(f"Artist ID {artist['id']} not found in artist CSV.")
                            continue
                        artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Create DataFrame and remove duplicates
artist_album_df = pd.DataFrame(artist_album_mapping).drop_duplicates()

# Save the DataFrame to CSV
artist_album_df.to_csv('artist_album_mappings_new.csv', index=False, sep=';')


In [ ]:
import pandas as pd
import os
import json

# Load the CSV files
albums_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\album_table.csv', delimiter=';', dtype={'id': str, 'album_spotify_id': str})
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\artists_data_full.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})

# Create dictionaries for ID mappings
album_id_map = albums_df.set_index('album_spotify_id')['id'].to_dict()
artist_id_map = artists_df.set_index('artist_spotify_id')['id'].to_dict()

artist_album_mapping = []
missing_artists = {}  # Store missing artist IDs and the album IDs they're attributed to

# Assuming the JSON structure is as shown in your example
for root, dirs, files in os.walk('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\ALBUMS'):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                data = json.load(f)
                for album in data['albums']:
                    album_id = album_id_map.get(album['id'])
                    if not album_id:
                        print(f"Album ID {album['id']} not found in album CSV.")
                        continue
                    for artist in album['artists']:
                        artist_id = artist_id_map.get(artist['id'])
                        if not artist_id:
                            # Record the missing artist ID along with the current album ID
                            missing_artists.setdefault(artist['id'], []).append(album['id'])
                            print(f"Artist ID {artist['id']} not found in artist CSV, attributed to Album ID {album['id']}.")
                            continue
                        artist_album_mapping.append({'artistID': artist_id, 'albumID': album_id})

# Display missing artist IDs and the albums they're attributed to
for artist_id, album_ids in missing_artists.items():
    print(f"Artist ID {artist_id} (not found) is attributed to Album IDs: {', '.join(album_ids)}")


In [ ]:
import pandas as pd

df1 = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\recordings_artist_list.csv', delimiter=',', dtype={'id': str, 'artist_spotify_id': str})
df2 = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\artists_table.csv', delimiter=';', dtype={'id': str, 'artist_spotify_id': str})


# Convert the musicbrainz_id column into a set for fast lookup
musicbrainz_ids = set(df2['musicbrainz_id'])

# Define a function to check if any artist_mbids match the musicbrainz_ids
def matches_musicbrainz_ids(artist_mbids_str):
    # Parse the string of artist_mbids into a list
    artist_mbids = artist_mbids_str.strip('{}').split(',')
    # Check if any of the artist_mbids is in the set of musicbrainz_ids
    return any(mbid.strip() in musicbrainz_ids for mbid in artist_mbids)

# Apply the function to filter df1
df1['artist_mbids_match'] = df1['artist_mbids'].apply(matches_musicbrainz_ids)
filtered_df1 = df1[df1['artist_mbids_match']]

# Drop the helper column after filtering
filtered_df1 = filtered_df1.drop(columns=['artist_mbids_match'])

# Save the filtered DataFrame to a new CSV file
output_csv_path = 'recordings_artist_list_onlyMatchedArtists.csv'
filtered_df1.to_csv(output_csv_path, index=False)

print(f"Filtered data saved to {output_csv_path}")


In [1]:
import pandas as pd

# Load CSV files into DataFrames
csv_path_1 = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\recordings_artist_list_onlyMatchedArtists.csv'
csv_path_2 = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\contributors_normalized.csv'

df1 = pd.read_csv(csv_path_1)
df2 = pd.read_csv(csv_path_2)

# Merge the DataFrames on 'recording_mbid'
# Ensure that the key for merging ('recording_mbid') exists in both DataFrames exactly as named.
merged_df = pd.merge(df1, df2, on='recording_mbid', how='inner')

# Select only the required columns
final_df = merged_df[['recording_mbid', 'recording_name_x', 'artist_names', 'artist_mbids', 'artist_mbid', 'individual_artist_name', 'role', 'instrument']]

# Optionally, rename 'recording_name_x' to 'recording_name' to correct the column name after merging
final_df.rename(columns={'recording_name_x': 'recording_name'}, inplace=True)

# Save the merged DataFrame to a new CSV file
output_csv_path = 'contributors_normalized_onlyMatchedArtists.csv'
final_df.to_csv(output_csv_path, index=False)

print(f"Combined data saved to {output_csv_path}")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
import pandas as pd

# Load the CSV files
contributors_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\WORKING\\contributors_normalized_onlyMatchedArtists.csv')
artists_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\artists_table.csv', sep=';')
artist_album_mapping_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\artist_album_mapping.csv', sep=';')
track_data_df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\tracks_data_full.csv', sep=';')

# Step 1: Merge artists with their albums
artists_with_albums = pd.merge(artists_df, artist_album_mapping_df, left_on='id', right_on='artistID', how='inner')

In [3]:
# Step 2: Merge the above with track data on albumID
artist_tracks = pd.merge(artists_with_albums, track_data_df, left_on='albumID', right_on='album_id', how='inner')

In [4]:
contributors_dict = contributors_df.groupby('artist_mbid')['recording_mbid'].apply(set).to_dict()


In [5]:
def map_recording_mbid(row):
    artist_mbids = contributors_dict.get(row['musicbrainz_id'], set())
    # Filter contributors where recording_name matches song_title and artist_mbid is in the dictionary entry
    for mbid in artist_mbids:
        # Filter by artist_mbid first to create a reduced subset
        possible_matches = contributors_df[(contributors_df['artist_mbid'] == mbid)]
        # Then check if recording_name matches song_title
        match = possible_matches[possible_matches['recording_name'] == row['song_title']]
        if not match.empty:
            return match['recording_mbid'].iloc[0]
    return None

# Apply the function across the artist_tracks DataFrame
artist_tracks['mapped_recording_mbid'] = artist_tracks.apply(map_recording_mbid, axis=1)


In [ ]:
final_tracks_with_mbid = artist_tracks.dropna(subset=['mapped_recording_mbid'])

# Save the final DataFrame to a new CSV file
final_tracks_with_mbid.to_csv('final_tracks_with_mapped_mbid.csv', sep=';', index=False)


In [ ]:
# Filter out rows where no recording_mbid was mapped (if necessary)
# final_tracks_with_mbid = artist_tracks.dropna(subset=['mapped_recording_mbid'])

# Save the final DataFrame to a new CSV file
final_tracks_with_mbid.to_csv('final_tracks_with_mapped_mbid.csv', sep=';', index=False)

print("Mapping complete and saved to final_tracks_with_mapped_mbid.csv")

In [1]:
file_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\tracks_data_full.csv'

In [2]:
import pandas as pd

df = pd.read_csv(file_path, delimiter=';')

# Remove rows where 'album_id' is empty
df_filtered = df[df['album_id'].notna()]

# Save the filtered DataFrame back to a new CSV file
output_file_path = 'tracks_data_full_new.csv'  # Replace with your desired output file path
df_filtered.to_csv(output_file_path, index=False, sep=';')

print(f"Filtered file saved to {output_file_path}")

Filtered file saved to tracks_data_full_new.csv


In [3]:
import pandas as pd

# Assuming you have already read the CSV into a DataFrame named df
df = pd.read_csv('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\SCRAPED_DATA\\FINISHED\\NEWEST\\tracks_data_full_new.csv', delimiter=';')

# Function to truncate strings longer than 254 characters
def truncate_string(s):
    return s[:254] if isinstance(s, str) and len(s) > 254 else s

# Apply the truncation function to the 'song_title' column
df['song_title'] = df['song_title'].apply(truncate_string)

# Save the DataFrame back to CSV
df.to_csv('tracks_data_full_new_new.csv', index=False, sep=';')